In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob
data_dir = Path("/content/drive/MyDrive")

beneficiary_files = sorted(glob.glob(str(data_dir/"beneficiary_*.csv")))
beneficiary_files

['/content/drive/MyDrive/beneficiary_2020.csv',
 '/content/drive/MyDrive/beneficiary_2021.csv',
 '/content/drive/MyDrive/beneficiary_2022.csv',
 '/content/drive/MyDrive/beneficiary_2023.csv']

In [3]:
#stack files into 1 df
def load_bene_file(path: str) -> pd.DataFrame:
  year_str = Path(path).stem.split("_")[-1]
  year = int(year_str)
  df = pd.read_csv( path, sep="|", dtype=str, low_memory=False) #load all as string to prevent removing data via autotype detection
  df["year"] = year #label year so we know which record came from which file
  return df

bene_list = [load_bene_file(p) for p in beneficiary_files]
bene_raw = pd.concat(bene_list, ignore_index=True)
bene_raw.shape, bene_raw.head()

((33933, 186),
            BENE_ID STATE_CODE COUNTY_CD ZIP_CD BENE_BIRTH_DT SEX_IDENT_CD  \
 0  -10000010254618         01      1500  36109   16-Aug-1999            1   
 1  -10000010254647         01      1410  35756   14-Aug-1990            1   
 2  -10000010254653         01      1400  36801   18-Mar-1982            2   
 3  -10000010254656         01      1360  35216   24-Jul-1999            2   
 4  -10000010254666         01      1440  35805   23-Feb-2009            2   
 
   BENE_RACE_CD ENTLMT_RSN_ORIG ENTLMT_RSN_CURR ESRD_IND  ...  \
 0            1               1               1        0  ...   
 1            2               1               1        0  ...   
 2            1               1               1        0  ...   
 3            1               1               1        0  ...   
 4            4               1               1        0  ...   
 
   CST_SHR_GRP_CD_04 CST_SHR_GRP_CD_05 CST_SHR_GRP_CD_06 CST_SHR_GRP_CD_07  \
 0                02                02       

In [4]:
#Create data dict from cms synthetic file documentation for pt summary
column_desc = {
  "BENE_ID": "Synthetic unique beneficiary identifier",
  "STATE_CODE": "Beneficiary state of residence",
  "COUNTY_CD": "Beneficiary county code within state",
  "ZIP_CD": "Beneficiary ZIP Code of residence",
  "BENE_BIRTH_DT": "Date of birth",
  "SEX_IDENT_CD": "Sex: 1=Male, 2=Female",
  "BENE_RACE_CD": (
        "Race code (Social Security Administration race classification): "
        "1=White, 2=Black, 3=Other, 4=Asian/Pacific Islander, "
        "5=Hispanic, 6=North American Native, 0=Unknown"
    ),
  "ENTLMT_RSN_ORIG": ("Original Medicare entitlement reason: 1=Age, 2=Disability, 3=ESRD, 4=Both disability & ESRD"),
  "ENTLMT_RSN_CURR": ("Current Medicare entitlement reason: 1=Age, 2=Disability, 3=ESRD, 4=Both disability & ESRD"),
  "ESRD_IND": "End-Stage Renal Disease indicator (1=ESRD beneficiary, 0=No)",
  "CST_SHR_GRP_CD_01": "Monthly cost-sharing group code (Jan)",
  "CST_SHR_GRP_CD_02": "Monthly cost-sharing group code (Feb)",
  "CST_SHR_GRP_CD_03": "Monthly cost-sharing group code (Mar)",
  "CST_SHR_GRP_CD_04": "Monthly cost-sharing group code (Apr)",
  "CST_SHR_GRP_CD_05": "Monthly cost-sharing group code (May)",
  "CST_SHR_GRP_CD_06": "Monthly cost-sharing group code (Jun)",
  "CST_SHR_GRP_CD_07": "Monthly cost-sharing group code (Jul)",
  "CST_SHR_GRP_CD_08": "Monthly cost-sharing group code (Aug)",
  "CST_SHR_GRP_CD_09": "Monthly cost-sharing group code (Sep)",
  "CST_SHR_GRP_CD_10": "Monthly cost-sharing group code (Oct)",
  "CST_SHR_GRP_CD_11": "Monthly cost-sharing group code (Nov)",
  "CST_SHR_GRP_CD_12": "Monthly cost-sharing group code (Dec)",
  "year": "File year"
}

bene_metadata = pd.DataFrame({
    "column": bene_raw.columns,
    "dtype": bene_raw.dtypes.astype(str),
    "description": bene_raw.columns.map(column_desc).fillna("")
})
bene_metadata.head()

,column,dtype,description
BENE_ID,BENE_ID,object,Synthetic unique beneficiary identifier
STATE_CODE,STATE_CODE,object,Beneficiary state of residence
COUNTY_CD,COUNTY_CD,object,Beneficiary county code within state
ZIP_CD,ZIP_CD,object,Beneficiary ZIP Code of residence
BENE_BIRTH_DT,BENE_BIRTH_DT,object,Date of birth


In [5]:
#create columns list for all billing files
var_desc = {
    #top level info
    "BENE_ID": "Synthetic unique beneficiary identifier",
    "CLM_ID": "Claim ID (encounter-level key)",
    "NCH_NEAR_LINE_REC_IDENT_CD": "NCH record type/ RIC (Part A/B, carrier, etc.)",
    "NCH_CLM_TYPE_CD": "NCH claim type code",
    "CLM_FROM_DT": "Claim from date",
    "CLM_THRU_DT": "Claim thru date",
    "NCH_WKLY_PROC_DT": "NCH weekly processing date",
    "FI_CLM_PROC_DT": "Intermediary /FI claim processing date",
    "CLAIM_QUERY_CODE": "Internal medicare claim query code/flag",
    #provider/facility
    "PRVDR_NUM": "Provider number",
    "ORG_NPI_NUM": "Organization NPI",
    "PRVDR_STATE_CD": "Provider state code",
    "PRVDR_ZIP": "Provider ZIP code",
    "PRVDR_SPCLTY": "Provider specialty code",
    "CARR_NUM": "Carrier number",
    "CARR_CLM_BLG_NPI_NUM": "Carrier billing NPI",
    #payment info
    "CLM_PMT_AMT": "Medicare payment amount for the claim",
    "NCH_PRMRY_PYR_CLM_PD_AMT": "Primary payer paid amount if not medicare",
    "NCH_CLM_PRVDR_PMT_AMT": "Provider payment amount medicare",
    "NCH_CLM_BENE_PMT_AMT": "Beneficiary payment amount",
    "CLM_TOT_CHRG_AMT": "Total charges for the claim",
    "NCH_IP_NCVRD_CHRG_AMT": "Non-covered charge amount for Part A or IP",
    "NCH_IP_TOT_DDCTN_AMT": "Total deductible/coinsurance amount for part A",
    #admits/discharge/utilization
    "CLM_ADMSN_DT": "Claim admission date",
    "CLM_IP_ADMSN_TYPE_CD": "Inpatient admission type code",
    "CLM_SRC_IP_ADMSN_CD": "Source of inpatient admission code",
    "PTNT_DSCHRG_STUS_CD": "Patient discharge status code",
    "NCH_PTNT_STATUS_IND_CD": "NCH patient status indicator",
    "CLM_NON_UTLZTN_DAYS_CNT": "Non covered utilization days on claim",
    "BENE_TOT_COINSRNC_DAYS_CNT": "Beneficiary total coinsurance days used",
    "BENE_LRD_USED_CNT": "Lifetime reserve days used",
    "NCH_BLOOD_PNTS_FRNSHD_QTY": "Blood units furnished in pints",
    "NCH_BENE_DSCHRG_DT": "Beneficiary discharge date for this stay",
    "CLM_DRG_CD": "Diagnosis Related Group",
    "CLM_DRG_OUTLIER_STAY_CD": "DRG outlier stay indicator",
    "NCH_DRG_OUTLIER_APRVD_PMT_AMT": "Approved DRG outlier payment amount",
    "CLM_PPS_IND_CD": "PPS indicator code",
    "CLM_TOT_PPS_CPTL_AMT": "Total PPS capital amount",
    "CLM_PPS_CPTL_DRG_WT_NUM": "PPS capital DRG weight",
    #diagnosis
    "ADMTG_DGNS_CD": "Admitting diagnosis code",
    "PRNCPAL_DGNS_CD": "Principal diagnosis code",
    "PRNCPAL_DGNS_VRSN_CD": "Principal diagnosis version (ICD-9/10)",
    #external injury flags
    "FST_DGNS_E_CD": "First external cause of injury code",
    "ICD_DGNS_E_CD1": "External cause of injury code 1",
    #code for procedures
    "ICD_PRCDR_CD1": "Inpatient ICD procedure code 1",
    "PRCDR_DT1": "Date for procedure code 1",
    #revenue centers or billing line specific fields (very granular)
    "CLM_LINE_NUM": "Sequential line number on the claim",
    "REV_CNTR": "Revenue center code",
    "REV_CNTR_DT": "Revenue center date of service",
    "REV_CNTR_UNIT_CNT": "Revenue center unit count",
    "REV_CNTR_RATE_AMT": "Revenue center rate amount",
    "REV_CNTR_TOT_CHRG_AMT": "Revenue center total charge amount",
    "REV_CNTR_NCVRD_CHRG_AMT": "Revenue center noncovered charge amount",
    "REV_CNTR_BENE_PMT_AMT": "Revenue center beneficiary payment amount",
    "REV_CNTR_PRVDR_PMT_AMT": "Revenue center provider payment amount",
    "REV_CNTR_PMT_AMT_AMT": "Revenue center total payment amount",
    "REV_CNTR_DDCTBL_COINSRNC_CD": "Revenue center deductible/coinsurance code",
    "REV_CNTR_NDC_QTY": "Revenue center NDC quantity",
    "REV_CNTR_NDC_QTY_QLFR_CD": "Revenue center NDC quantity qualifier",
    "REV_CNTR_APC_HIPPS_CD": "Revenue center APC/HIPPS code",
    "REV_CNTR_PMT_MTHD_IND_CD": "Revenue center payment method indicator",
    "REV_CNTR_STUS_IND_CD": "Revenue center status indicator",
    "REV_CNTR_DSCNT_IND_CD": "OPPS discount indicator code",
    "REV_CNTR_OTAF_PMT_CD": "Obligation to accept as full payment code",
    "REV_CNTR_PACKG_IND_CD": "Packaging indicator",
    #HCPCS codes/mods
    "HCPCS_CD": "HCPCS/CPT procedure code",
    "HCPCS_1ST_MDFR_CD": "HCPCS first modifier",
    "HCPCS_2ND_MDFR_CD": "HCPCS second modifier",
    "HCPCS_3RD_MDFR_CD": "HCPCS third modifier",
    "HCPCS_4TH_MDFR_CD": "HCPCS fourth modifier",
    #line fields for carrier/dme
    "LINE_NUM": "Line number",
    "LINE_SRVC_CNT": "Line service count",
    "LINE_1ST_EXPNS_DT": "First expense date for the line",
    "LINE_LAST_EXPNS_DT": "Last expense date for the line",
    "LINE_NCH_PMT_AMT": "Line-level Medicare payment amount",
    "LINE_BENE_PMT_AMT": "Line-level beneficiary payment amount",
    "LINE_PRVDR_PMT_AMT": "Line-level provider payment amount",
    "LINE_BENE_PTB_DDCTBL_AMT": "Line-level Part B deductible amount",
    "LINE_COINSRNC_AMT": "Line-level coinsurance amount",
    "LINE_SBMTD_CHRG_AMT": "Line submitted charge amount",
    "LINE_ALOWD_CHRG_AMT": "Line allowed charge amount",
    "LINE_PMT_80_100_CD": "Line payment 80/100 percent indicator",
    "LINE_CMS_TYPE_SRVC_CD": "HCFA/CMS line type of service",
    "LINE_PLACE_OF_SRVC_CD": "Line place of service code",
    "LINE_SERVICE_DEDUCTIBLE": "Indicator if line is subject to deductible",
    "LINE_NDC_CD": "Line National Drug Code",
    "LINE_HCT_HGB_RSLT_NUM": "Hematocrit/hemoglobin lab result value",
    "LINE_HCT_HGB_TYPE_CD": "Hematocrit/hemoglobin result type code",
    "LINE_ICD_DGNS_CD": "Line level diagnosis code",
    "LINE_ICD_DGNS_VRSN_CD": "Line-level diagnosis version (ICD-9/10)",
    #hospice/hha/snf specific unique fields
    "CLM_HHA_LUPA_IND_CD": "HHA low utilization payment adjustment indicator",
    "CLM_HHA_RFRL_CD": "HHA referral code",
    "CLM_HHA_TOT_VISIT_CNT": "Total HHA visits in the episode",
    "CLM_HOSPC_START_DT_ID": "Hospice start date for claim",
    "BENE_HOSPC_PRD_CNT": "Beneficiary hospice period count",
    "CLM_UTLZTN_DAY_CNT": "Covered hospice/SNF utilization days",
    "NCH_QLFYD_STAY_FROM_DT": "Qualified SNF stay from date",
    "NCH_QLFYD_STAY_THRU_DT": "Qualified SNF stay thru date",
    #phyysician visits
    "AT_PHYSN_UPIN": "Attending physician UPIN",
    "AT_PHYSN_NPI": "Attending physician NPI",
    "OP_PHYSN_UPIN": "Operating physician UPIN",
    "OP_PHYSN_NPI": "Operating physician NPI",
    "OT_PHYSN_UPIN": "Other physician UPIN",
    "OT_PHYSN_NPI": "Other physician NPI",
    "RFR_PHYSN_UPIN": "Referring physician UPIN",
    "RFR_PHYSN_NPI": "Referring physician NPI",
    "PRF_PHYSN_UPIN": "Performing physician UPIN",
    "PRF_PHYSN_NPI": "Performing physician NPI",
    "RNDRNG_PHYSN_UPIN": "Rendering physician UPIN",
    "RNDRNG_PHYSN_NPI": "Rendering physician NPI",
}
#add repeating code columns Im not adding manually
for i in range (1, 26):
  var_desc[f"ICD_DGNS_CD{i}"] = f"Claim diagnosis code {i} (ICD-9/ICD-10)"
  var_desc[f"ICD_DGNS_E_CD{i}"] = f"External cause of injury code {i}"
  var_desc[f"ICD_PRCDR_CD{i}"] = f"Procedure code {i} (ICD-9/ICD-10)"
  var_desc[f"PRCDR_DT{i}"] = f"Date of Procedure {i}"

for i in range (1, 4):
  var_desc[f"RSN_VISIT_CD{i}"] = f"Reason for visit diagnosis code{i}"

In [6]:
#funct to match header descriptions
def load_claims_labels(path, var_desc_dict):
  df = pd.read_csv(path,sep="|", dtype=str)
  labels = {col:var_desc_dict.get(col, "") for col in df.columns}
  df.attrs["variable_lables"] = labels
  return df


In [7]:
#define dataframes + labels
inpatient = load_claims_labels("/content/drive/MyDrive/inpatient.csv", var_desc)
outpatient = load_claims_labels("/content/drive/MyDrive/outpatient.csv", var_desc)
snf = load_claims_labels("/content/drive/MyDrive/snf.csv", var_desc)
hospice = load_claims_labels("/content/drive/MyDrive/hospice.csv", var_desc)
hha = load_claims_labels("/content/drive/MyDrive/hha.csv", var_desc)
carrier = load_claims_labels("/content/drive/MyDrive/carrier.csv", var_desc)
dme = load_claims_labels("/content/drive/MyDrive/dme.csv", var_desc)

In [8]:
inpatient.head()

,BENE_ID,CLM_ID,NCH_NEAR_LINE_REC_IDENT_CD,NCH_CLM_TYPE_CD,CLM_FROM_DT,CLM_THRU_DT,NCH_WKLY_PROC_DT,FI_CLM_PROC_DT,CLAIM_QUERY_CODE,PRVDR_NUM,...,PRCDR_DT24,ICD_PRCDR_CD25,PRCDR_DT25,IME_OP_CLM_VAL_AMT,DSH_OP_CLM_VAL_AMT,CLM_UNCOMPD_CARE_PMT_AMT,CLM_LINE_NUM,REV_CNTR,HCPCS_CD,REV_CNTR_DDCTBL_COINSRNC_CD
0,-10000010254618,-10000930037831,V,60,25-Mar-2015,25-Mar-2015,27-Mar-2015,NaN,3,011500,...,NaN,NaN,NaN,0,0,NaN,1,0450,99221,NaN
1,-10000010254653,-10000930038030,V,60,24-Sep-2015,24-Sep-2015,25-Sep-2015,NaN,3,017129,...,NaN,NaN,NaN,0,0,NaN,1,0450,99221,NaN
2,-10000010254653,-10000930038031,V,60,09-May-2017,10-May-2017,12-May-2017,NaN,3,010052,...,NaN,NaN,NaN,0,0,NaN,1,0001,99024,3
3,-10000010254656,-10000930038162,V,60,14-Jan-2017,14-Jan-2017,20-Jan-2017,NaN,3,015455,...,NaN,NaN,NaN,0,0,NaN,1,0450,73610,3
4,-10000010254656,-10000930038162,V,60,14-Jan-2017,14-Jan-2017,20-Jan-2017,NaN,3,015455,...,NaN,NaN,NaN,0,0,NaN,2,0450,29515,3


In [9]:
#the remaining data needs to be normalized and joined
import re
def clean_facility_id(x):
  if pd.isna(x): return None
  s = re.sub(r"\D", "", str(x))
  return s.zfill(6) if s else None
def coerce_zip(x):
  if pd.isna(x): return None
  s = re.sub(r"\D", "", str(x))
  if len(s) >= 5:
    return s[:5] if len(s) == 5 or s[:5] != "00000" else s[-5:]
  return None
def timeframe_to_year(tf):
  if pd.isna(tf): return None
  m = re.findall(r"\d{4}", str(tf))
  return int(m[-1]) if m else None

def latest_per_zip(df, time_col="timeframe", zip_col="zip"):
  d = df.copy()
  d["_year"] = d[time_col].map(timeframe_to_year)
  d.sort_values([zip_col, "_year"], inplace=True)
  return d.groupby(zip_col, as_index=False).tail(1).drop(columns=["_year"])

if {"BENE_ID", "ZIP_CD","year"}.issubset(bene_raw.columns):
  bene_latest = (
      bene_raw.assign(zip=bene_raw["ZIP_CD"]
                      .map(coerce_zip))
                      .sort_values(["BENE_ID","year"])
                      .groupby("BENE_ID",as_index=False)
                      .tail(1)[["BENE_ID", "zip", "STATE_CODE","year"]]
  )
else:
  bene_latest = pd.DataFrame(columns=["BENE_ID","zip","STATE_CODE","year"])

In [10]:
#load cms gen info and cms admit reduction files and clean
cms_gen = pd.read_csv(
    "/content/drive/MyDrive/CMSCompareHospitalGeneralInformation.csv",
    sep=",", engine="python", dtype="string"
    )
cms_gen = cms_gen.rename(columns={"Facility ID": "facility_id",
                                  "Facility Name": "facility_name",
                                  "State": "state"
                                  })
cms_gen["facility_id"] = cms_gen["facility_id"].map(clean_facility_id)
cms_gen = cms_gen.dropna(subset=["facility_id"]).drop_duplicates("facility_id")

cms_readm = pd.read_csv(
    "/content/drive/MyDrive/CMSCompareHospitalReadmissionReduction.csv",
    sep=",", engine="python", dtype="string",
)

cms_readm - cms_readm.rename(columns={
    "Facility ID": "facility_id",
    "Measure Name": "measure",
    "Expected Readmission Rate": "expected_rate",
    "Number of Readmissions": "readmit_count"
}, inplace=True)
cms_readm["facility_id"] = cms_readm["facility_id"].map(clean_facility_id)
cms_readm["expected_rate"] = pd.to_numeric(cms_readm["expected_rate"], errors="coerce")
cms_readm["readmit_count"] = pd.to_numeric(cms_readm["readmit_count"], errors="coerce")

measure_map ={
    "READM-30-AMI-HRRP": "readm_expected_rate_ami",
    "READM-30-CABG-HRRP": "readm_expected_rate_cabg",
    "READM-30-HF-HRRP": "readm_expected_rate_hf",
    "READM-30-HIP-KNEE-HRRP": "readm_expected_rate_hip_knee",
    "READM-30-PN-HRRP": "readm_expected_rate_pn",
    "READM-30-COPD-HRRP": "readm_expected_rate_copd",

}
cms_readm = cms_readm[cms_readm["measure"].isin(measure_map)].copy()
cms_readm["measure_col"] = cms_readm["measure"].map(measure_map)

rates = (cms_readm.pivot_table(index="facility_id", columns="measure_col",
                              values="expected_rate", aggfunc="mean").reset_index()
                              )
counts = (cms_readm.pivot_table(index="facility_id", columns="measure_col",
                              values="readmit_count", aggfunc="sum").add_suffix("_count").reset_index()
                              )
facility_features = cms_gen.merge(rates, on="facility_id", how="left").merge(counts, on="facility_id", how="left")


In [11]:
#load policy map files and clean

policymap_common_cols = {
    "GeoID_Description","GeoID_Name","SitsinState","GeoID","GeoID_Formatted", "TimeFrame","GeoVintage","Source","Location"
}
def load_all_policymap(base_dir: Path = Path("/content/drive/MyDrive")):
  chunks = []
  for p in sorted(base_dir.glob("PolicyMap*.csv")):
    pm = pd.read_csv(p, header=1, dtype="string", low_memory=False)
    value_cols = [c for c in pm.columns if c not in policymap_common_cols] #pick the column thats different from file to file
    value_col = value_cols[0] if value_cols else pm.columns[5]
    pm = pm.rename(columns={"SitsinState":"state","TimeFrame":"timeframe"})
    pm["zip"]= (pm["GeoID_Name"].map(coerce_zip).fillna(pm["GeoID"].fillna(pm["GeoID_Formatted"].map(coerce_zip))))
    pm = pm[pm["zip"].str.len() ==5].copy()
    feature_name = "policymap_" + p.stem.replace("PolicyMap", "").lower() + "_pct"
    pm = pm [["zip", "state", "timeframe", value_col]].rename(columns={value_col: feature_name})
    pm = latest_per_zip(pm, time_col ="timeframe", zip_col="zip")[["zip", feature_name]]
    chunks.append(pm)

  if not chunks: return pd.DataFrame(columns=["zip"])
  out = chunks[0]
  for c in chunks[1:]:
    out = out.merge(c, on="zip", how="outer")
  return out
policymap_wide= load_all_policymap(Path("/content/drive/MyDrive"))

In [12]:
#load cdc places file and clean

cdc = pd.read_csv("/content/drive/MyDrive/CdcPlaces.csv", dtype="string", low_memory=False)
cdc["zip"] = cdc["LocationID"].map(coerce_zip)
cdc = cdc[cdc["zip"].str.len() ==5].copy()

cdc["Year"] = pd.to_numeric(cdc["Year"], errors="coerce")
cdc["Data_Value"] = pd.to_numeric(cdc["Data_Value"], errors="coerce")

#group chronic measures
chronic_measures = [
  "Obesity among adults",
  "Chronic obstructive pulmonary disease among adults",
  "Diagnosed diabetes among adults",
  "Coronary heart disease among adults",
  "High blood pressure among adults",
  "High cholesterol among adults who have ever been screened",
  "Current asthma among adults",
  "Cancer (non-skin) or melanoma among adults",
  "Current cigarette smoking among adults",
  "Arthritis among adults"
]
#group poor physical health
poor_physical_measures =[
  "Frequent physical distress among adults",
  "Frequent mental distress among adults",
  "Fair or poor self-rated health status among adults",
  "Mobility disability among adults",
  "Vision disability among adults",
  "Hearing disability among adults",
  "All teeth lost among adults aged >=65 years",
  "Any disability among adults",
  "Independent living disability among adults",
  "Short sleep duration among adults",
  "Lack of social and emotional support among adults",
  "Feeling socially isolated among adults"
]
#group preventative care
preventative_measures = [
  "Visits to doctor for routine checkup within the past year among adults",
  "Cholesterol screening among adults",
  "Colorectal cancer screening among adults aged 45–75 years",
  "Mammography use among women aged 50-74 years",
  "Visited dentist or dental clinic in the past year among adults",
  "Taking medicine to control high blood pressure among adults with high blood pressure"
]
m2g = {m: "chronic_conditions_pct" for m in chronic_measures}
m2g.update({m: "poor_physical_health_pct" for m in poor_physical_measures})
m2g.update({m: "preventative_care_pct" for m in preventative_measures})

cdc = cdc[cdc["Measure"].isin(m2g)]
cdc["Data_Value"] = pd.to_numeric(cdc["Data_Value"], errors="coerce")
cdc["Year"] = pd.to_numeric(cdc["Data_Value"], errors="coerce")
cdc["weight"] = pd.to_numeric(cdc.get("TotalPop18plus"), errors="coerce").fillna(pd.to_numeric(cdc.get("TotalPopulation"), errors="coerce"))

idx = cdc.groupby(["zip", "Measure"])["Year"].idxmax()
cdc= cdc.loc[idx].copy()
cdc["group"] = cdc["Measure"].map(m2g)

agg = (cdc.assign(w=cdc["Data_Value"] *  cdc["weight"])
.groupby(["zip","group"], as_index=False)
.agg(total_weight=("weight","sum"), sum_weighted=("w","sum"))
)

agg["val"] = agg["sum_weighted"]/ agg["total_weight"]
cdc_grouped = agg.pivot(index="zip", columns="group", values="val").reset_index()
for col in ("chronic_conditions_pct", "poor_physical_health_pct", "preventative_care_pct"):
  if col not in cdc_grouped.columns:
    cdc_grouped[col] = np.nan


In [13]:
#enrich claims based on SDOH data sets and other medical, bringing together everything
def enrich_claims(df, attach_facility=True):
  df = df.merge(bene_latest[["BENE_ID", "zip"]], on="BENE_ID", how="left")
  if attach_facility:
    df = df.assign(facility_id=df["PRVDR_NUM"].map(clean_facility_id))
    df = df.merge(facility_features, on="facility_id", how="left")
  df = df.merge(policymap_wide, on="zip", how="left")
  df = df.merge(cdc_grouped, on="zip", how="left")
  return df

inpatient_enriched = enrich_claims(inpatient, attach_facility=True)
outpatient_enriched = enrich_claims(outpatient, attach_facility=True)
snf_enriched = enrich_claims(snf, attach_facility=True)
hha_enriched = enrich_claims(hha, attach_facility=True)
hospice_enriched = enrich_claims(hospice, attach_facility=True)
#these ***shouldn't have a facility
carrier_enriched = enrich_claims(carrier, attach_facility=False)
dme_enriched = enrich_claims(dme, attach_facility=False)

In [14]:
#prepping data for gbdt training

ip = inpatient_enriched.copy()
ip["admit_dt"] = pd.to_datetime(ip["CLM_ADMSN_DT"], errors="coerce")
ip["dschrg_dt"] = pd.to_datetime(ip["NCH_BENE_DSCHRG_DT"], errors="coerce")
ip = ip[~ip["admit_dt"].isna() & ~ip["dschrg_dt"].isna()].copy()

ip = ip[ip["PTNT_DSCHRG_STUS_CD"] != "20"].copy()
ip.sort_values(["BENE_ID", "admit_dt", "dschrg_dt", "CLM_ID"], inplace=True)
ip["next_admit_dt"] = ip.groupby("BENE_ID")["admit_dt"].shift(-1)
ip["days_to_next_admit"] = (ip["next_admit_dt"] - ip["dschrg_dt"]).dt.days
ip["readmit_30"] = ((ip["days_to_next_admit"] > 0) & (ip["days_to_next_admit"] <=30)).astype("int8")

def _prior_counts(g, window_days: int):
  arr = g["admit_dt"].values.astype("datetime64[D]").astype("int64")
  out = np.zeros(len(arr), dtype="int16")
  for i in range (len(arr)):
    cutoff = arr[i] - window_days
    j = np.searchsorted(arr, cutoff, side="left")
    out[i] = i - j
  return pd.Series(out, index=g.index)

ip["prior_admits_180d"] = ip.groupby("BENE_ID", group_keys=False).apply(lambda g: _prior_counts(g, 180)).astype("int16")
ip["prior_admits_365d"] = ip.groupby("BENE_ID", group_keys=False).apply(lambda g: _prior_counts(g, 365)).astype("int16")

ip["los_days"] = (ip["dschrg_dt"] - ip["admit_dt"]).dt.days.clip(lower=0).astype("int16")

bene_demo = ( bene_raw[["BENE_ID", "BENE_BIRTH_DT", "SEX_IDENT_CD", "BENE_RACE_CD"]].drop_duplicates("BENE_ID", keep="last").copy())
bene_demo["birth_dt"] = pd.to_datetime(bene_demo["BENE_BIRTH_DT"], errors="coerce")

ip = ip.merge(bene_demo[["BENE_ID", "birth_dt","SEX_IDENT_CD", "BENE_RACE_CD"]], on="BENE_ID", how="left")
ip["age"] = ((ip["admit_dt"] - ip["birth_dt"]).dt.days / 365.25).astype("float32")
ip["sex"] = pd.to_numeric(ip["SEX_IDENT_CD"], errors="coerce").astype("float32")
ip["race"] = pd.to_numeric(ip["BENE_RACE_CD"], errors="coerce").astype("float32")

for c in ["CLM_TOT_CHRG_AMT", "CLM_PMT_AMT"]:
  ip[c] = pd.to_numeric(ip[c], errors="coerce")
  ip[f"log_{c.lower()}"] = np.log1p(ip[c]).astype("float32")

for c in ["CLM_DRG_CD", "CLM_IP_ADMSN_TYPE_CD", "CLM_SRC_IP_ADMSN_CD","PTNT_DSCHRG_STUS_CD"]:
  ip[c] = pd.to_numeric(ip[c], errors="coerce") #.astype("int16") left as float as gbdt's handle float nans

dx_cols = [c for c in ip.columns if c.startswith("ICD_DGNS_CD")]
ip["dx_count"] = ip[dx_cols]. notna().sum(axis=1).astype("int16")
id_cols = ["BENE_ID", "CLM_ID", "facility_id", "zip", "admit_dt", "dschrg_dt"]
label_col = "readmit_30"

numeric_cols = [c for c in ip.columns if c not in id_cols and c != label_col and not c.startswith("ICD_") and pd.api.types.is_numeric_dtype(ip[c])]

train_inpatient = ip[id_cols + [label_col] + numeric_cols].copy()
#fill NA's median
feature_medians = train_inpatient[numeric_cols].median()
X_gbdt = train_inpatient[numeric_cols].fillna(feature_medians)
y_gbdt = train_inpatient[label_col].astype("int8")

keys_gbdt = train_inpatient[id_cols].copy()




/tmp/ipython-input-3225719694.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ip["prior_admits_180d"] = ip.groupby("BENE_ID", group_keys=False).apply(lambda g: _prior_counts(g, 180)).astype("int16")
/tmp/ipython-input-3225719694.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ip["prior_admits_365d"] = ip.groupby("BENE_ID", group_keys=False).apply(lambda g: _prior_counts(g, 365)).astype("int16")


In [15]:
!pip install catboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.8 MB/s eta 0:00:00


In [16]:
#training models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, brier_score_loss, accuracy_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer # last minute addition

df = train_inpatient.copy()
target_col = "readmit_30"
id_cols = [c for c in ["BENE_ID", "CLM_ID", "PROVNUM", "PRVDR_NUM"] if c in df.columns]

#2nd last minute fix as lightgbm seems to hate object datatypes
for col in ["facility_id", "zip"]:
  if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

y = df[target_col].astype(int)
X = df.drop(columns=[target_col] + id_cols) #need to remove primary/foreign key columns to prevent models from cheating via ID's
if "days_to_next_admit" in X.columns:
  X = X.drop(columns=["days_to_next_admit"])

# need to drop datetime columns as we cant cast datetime to float64 for model training
dt_cols = X.select_dtypes(include=["datetime64[ns]", "datetimetz"]).columns.tolist()
date_like_cols = [c for c in X.columns if "DT" in c or "DATE" in c.upper() or "DT" in c.upper()]
X = X.drop(columns=list(set(dt_cols + date_like_cols)), errors="ignore")

X = X.loc[:, X.nunique(dropna=False) > 1]

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state=42, stratify=y)

#logistic regression ~Linear was going to be VERY tedious
log_clf = Pipeline([("imputer", SimpleImputer(strategy="median")), #impute due to logistic model stopping everything from NaNs
 ("scaler", StandardScaler(with_mean=False)),
  ("logreg", LogisticRegression(max_iter=200, n_jobs=-1))])
#LightGBM
lgbm_clf = LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=-1, subsample=0.8, colsample_bytree=0.8, randomstate=42)
#CatBoost
cat_clf = CatBoostClassifier(iterations=300, learning_rate=0.05, depth=6, loss_function="Logloss", verbose=False, random_state=42)
models = {"Logistic Regression": log_clf, "LightGBM":lgbm_clf, "CatBoost": cat_clf}

def eval_model(name, model, X_train, y_train, X_test, y_test):
  model.fit(X_train, y_train)
  if hasattr(model, "predict_proba"):
    y_prob = model.predict_proba(X_test)[:, 1]
  else:
    y_prob = model.predict(X_test)

  y_pred = (y_prob >=0.5).astype(int)

  auc = roc_auc_score(y_test, y_prob)
  brier = brier_score_loss(y_test, y_prob)
  acc = accuracy_score(y_test, y_pred)

  print(f"ROC AUC: {auc:0.4f}")
  print(f"Brier Score: {brier:0.4f}")
  print(f"Accuracy: {acc:0.4f}")

for name, model in models.items():
  eval_model(name, model, X_train, y_train, X_test, y_test)

ROC AUC: 0.7855
Brier Score: 0.0825
Accuracy: 0.8978
[LightGBM] [Warning] Unknown parameter: randomstate
[LightGBM] [Warning] Unknown parameter: randomstate
[LightGBM] [Info] Number of positive: 4868, number of negative: 41584
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039520 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4474
[LightGBM] [Info] Number of data points in the train set: 46452, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.104796 -> initscore=-2.145032
[LightGBM] [Info] Start training from score -2.145032
[LightGBM] [Warning] Unknown parameter: randomstate
ROC AUC: 0.8431
Brier Score: 0.0697
Accuracy: 0.9149
ROC AUC: 0.8426
Brier Score: 0.0709
Accuracy: 0.9123


In [17]:
!pip install shap

In [18]:
#impliment shap & first report export

import shap
export_dir = Path("/content/drive/MyDrive/exports")
export_dir.mkdir(exist_ok=True)

results =[]
probs_test = pd.DataFrame(index=X_test.index)

for name, model in models.items():
  model.fit(X_train, y_train)
  if hasattr(model, "predict_proba"):
    y_prob = model.predict_proba(X_test)[:, 1]
  else:
    y_prob = model.predict(X_test)

  y_pred = (y_prob >=0.5).astype(int)

  auc = roc_auc_score(y_test, y_prob)
  brier = brier_score_loss(y_test, y_prob)
  acc = accuracy_score(y_test, y_pred)
  results.append({"model": name,
                  "roc_auc": auc,
                  "Brier_score":brier,
                  "accuracy":acc})
  short = name.lower().replace(" ", "_")
  probs_test[f"pred_{short}"] = y_prob


model_performance = pd.DataFrame(results)
model_performance.to_csv(export_dir / "model_performance.csv", index=False)
model_performance


[LightGBM] [Warning] Unknown parameter: randomstate
[LightGBM] [Warning] Unknown parameter: randomstate
[LightGBM] [Info] Number of positive: 4868, number of negative: 41584
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014936 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4474
[LightGBM] [Info] Number of data points in the train set: 46452, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.104796 -> initscore=-2.145032
[LightGBM] [Info] Start training from score -2.145032
[LightGBM] [Warning] Unknown parameter: randomstate


,model,roc_auc,Brier_score,accuracy
0,Logistic Regression,0.785489,0.082477,0.897796
1,LightGBM,0.843101,0.069731,0.914930
2,CatBoost,0.842583,0.070899,0.912261


In [19]:
#make predictions with test data, target, and model probs

test_ids = pd.DataFrame(index=X_test.index)
id_cols_available = [c for c in ["BENE_ID", "CLM_ID", "PROVNUM", "PRVDR_NUM"] if c in df.columns]

if id_cols_available:
  test_ids[id_cols_available] = df.loc[X_test.index, id_cols_available]

test_predictions = test_ids.copy()
test_predictions["readmit_30"] = y_test.values
test_predictions = test_predictions.join(probs_test)
test_predictions.to_csv(export_dir / "test_predictions.csv", index=False)
test_predictions.head()


,BENE_ID,CLM_ID,readmit_30,pred_logistic_regression,pred_lightgbm,pred_catboost
36112,-10000010274441,-10000930921224,0,0.043127,0.061420,0.050793
30334,-10000010271183,-10000930767610,1,0.162994,0.216139,0.186748
57412,-10000010287504,-10000931457779,0,0.065869,0.017931,0.037189
17696,-10000010263715,-10000930446843,0,0.080111,0.136928,0.144116
16784,-10000010263014,-10000930414590,0,0.008386,0.004969,0.016698


In [20]:
#feature importance for GBDT Models

feat_names = X_train.columns.tolist()
lgbm_importance = pd.DataFrame({"feature": feat_names, "importance": lgbm_clf.feature_importances_}).sort_values("importance", ascending=False)
lgbm_importance.to_csv(export_dir / "Lgbm_feature_importance.csv", index=False)


cat_importances_vals = cat_clf.get_feature_importance()
cat_importances = pd.DataFrame({"feature": feat_names, "importances": cat_importances_vals}).sort_values("importances", ascending=False)

cat_importances.to_csv(export_dir / "catboost_feature_importance.csv", index=False)

lgbm_importance.head(), cat_importances.head()


(              feature  importance
 2         CLM_PMT_AMT        1352
 22  prior_admits_180d        1218
 23  prior_admits_365d        1115
 25                age        1009
 0         facility_id         716,
                  feature  importances
 22     prior_admits_180d    14.533547
 29       log_clm_pmt_amt    10.495866
 23     prior_admits_365d    10.109625
 28  log_clm_tot_chrg_amt     8.514231
 2            CLM_PMT_AMT     7.995051)

In [21]:
#sjhap for lightbgm summary and long format sample

#funct for converting pandas NAtype to numpy's nan for shap
def to_numpy_float(df):
  return df.astype("float64").replace({pd.NA: np.nan}).to_numpy()

#decrease sample size to prevent hours of runtime
n_bg = min(5000, len(X_train))
X_bg = X_train.sample(n=n_bg, random_state=42)

n_shap = min(2000, len(X_test))
X_test_shap = X_test.sample(n=n_shap, random_state=42)
#convert to numpy
X_bg_np = to_numpy_float(X_bg)
X_test_shap_np = to_numpy_float(X_test_shap)

#Treeexplainer for lgbm
explainer_lgbm = shap.TreeExplainer(lgbm_clf)
shap_values = explainer_lgbm.shap_values(X_test_shap_np)

#to handle weird list shap output
if isinstance(shap_values, list):
  shap_vals = shap_values[1]
else:
  shap_vals = shap_values

#shap mean for each feature
mean_abs_shap = np.abs(shap_vals).mean(axis=0)
shap_summary_lgbm = pd.DataFrame({"feature": X_test_shap.columns,
                                  "mean_abs_shap": mean_abs_shap}).sort_values("mean_abs_shap",ascending=False)


shap_summary_lgbm.to_csv(export_dir / "shap_summary_lgbm.csv",index=False)
shap_summary_lgbm.head()


/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


,feature,mean_abs_shap
2,CLM_PMT_AMT,0.772628
22,prior_admits_180d,0.592359
23,prior_admits_365d,0.194413
24,los_days,0.176549
30,dx_count,0.166949


In [22]:
#shap long format for bi visuals

n_rows_long = min(100, len(X_test_shap))
X_long= X_test_shap.iloc[:n_rows_long]
shap_long_vals = shap_vals[:n_rows_long, :]

records = []
for i, (idx, row) in enumerate(X_long.iterrows()):
  for j, feat in enumerate(X_long.columns):
                           records.append({"row_id":idx,"feature":feat,"feature_value":row[feat],"shap_value":shap_long_vals[i,j]
                                           })


shap_sample_long = pd.DataFrame(records)
#join beneficiary id onto shap samples for possible drilldown
if "BENE_ID" in df.columns:
  shap_sample_long["BENE_ID"] = shap_sample_long["row_id"].map(df["BENE_ID"])

shap_sample_long.to_csv(export_dir / "shap_sample_lgbm.csv", index=False)

shap_sample_long.head()


,row_id,feature,feature_value,shap_value,BENE_ID
0,30697,facility_id,33394.0,0.035480,-10000010271384
1,30697,zip,13905,0.081425,-10000010271384
2,30697,CLM_PMT_AMT,379024.23,-1.414248,-10000010271384
3,30697,CLM_TOT_CHRG_AMT,379024.23,-0.193066,-10000010271384
4,30697,CLM_IP_ADMSN_TYPE_CD,3,-0.036450,-10000010271384


In [23]:
#Patient level shap example for example report printable

pred_lgbm_test = probs_test["pred_lightgbm"] # bring in previous predicted probabilities
#grab top 100 readmit risk patients
high_risk_idx = pred_lgbm_test.sort_values(ascending=False).head(100).index

X_patients = X_test.loc[high_risk_idx]
shap_patients = explainer_lgbm.shap_values(X_patients)

if isinstance(shap_patients, list):
  shap_pat_vals = shap_patients[1]
else:
  shap_pat_vals = shap_patients


patient_records = []
for i, (idx, row) in enumerate(X_patients.iterrows()):
  for j, feat in enumerate(X_patients.columns):
    rec = {"row_id":idx,"feature":feat,
           "feature_value":row[feat],
           "shap_value":shap_pat_vals[i,j],
           "pred_prob_lgbm":pred_lgbm_test.loc[idx],
           "actual_readmit_30":int(y_test.loc[idx])
           }
    if "BENE_ID" in df.columns:
      rec["BENE_ID"] = df.loc[idx, "BENE_ID"]
    if "CLM_ID" in df.columns:
      rec["CLM_ID"] = df.loc[idx, "CLM_ID"]
    patient_records.append(rec)


shap_patient_detail = pd.DataFrame(patient_records)
shap_patient_detail.to_csv(export_dir / "shap_patient_detail_lgbm.csv", index=False)
shap_patient_detail.head()

/usr/local/lib/python3.12/dist-packages/shap/explainers/_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


,row_id,feature,feature_value,shap_value,pred_prob_lgbm,actual_readmit_30,BENE_ID,CLM_ID
0,38037,facility_id,377673.0,-0.027021,0.99638,1,-10000010275202,-10000930959155
1,38037,zip,73012,-0.015331,0.99638,1,-10000010275202,-10000930959155
2,38037,CLM_PMT_AMT,726.75,2.772036,0.99638,1,-10000010275202,-10000930959155
3,38037,CLM_TOT_CHRG_AMT,726.75,0.320481,0.99638,1,-10000010275202,-10000930959155
4,38037,CLM_IP_ADMSN_TYPE_CD,1,0.036532,0.99638,1,-10000010275202,-10000930959155
